# Imports

In [1]:
import os
import sys

# root path
ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(ROOT)

In [10]:
import polars as pl
from src.config import CONFIG

RAW_DATA_PATH = CONFIG.RAW_DATA_PATH
PRO_DATA_PATH_v1 = CONFIG.PRO_DATA_PATH_v1

# Data load and downcast

In [4]:
df = pl.read_csv(RAW_DATA_PATH, try_parse_dates=True)

In [5]:
df.shape

(420551, 15)

In [6]:
df

Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2009-01-01 00:10:00,996.52,-8.02,265.4,-8.9,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.8,0.72,1.5,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.2,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.5,198.0
2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.0,0.32,0.63,214.3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2016-12-31 23:20:00,1000.07,-4.05,269.1,-8.13,73.1,4.52,3.3,1.22,2.06,3.3,1292.98,0.67,1.52,240.0
2016-12-31 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
2016-12-31 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.0,215.2


In [9]:
print(df.to_pandas().describe())

                        Date Time       p (mbar)       T (degC)  \
count                      420551  420551.000000  420551.000000   
mean   2012-12-30 06:26:51.498724     989.212776       9.450147   
min           2009-01-01 00:10:00     913.600000     -23.010000   
25%           2010-12-31 03:25:00     984.200000       3.360000   
50%           2012-12-30 06:20:00     989.580000       9.420000   
75%           2014-12-29 18:55:00     994.720000      15.470000   
max           2017-01-01 00:00:00    1015.350000      37.280000   
std                           NaN       8.358481       8.423365   

            Tpot (K)    Tdew (degC)         rh (%)   VPmax (mbar)  \
count  420551.000000  420551.000000  420551.000000  420551.000000   
mean      283.492743       4.955854      76.008259      13.576251   
min       250.600000     -25.010000      12.950000       0.950000   
25%       277.430000       0.240000      65.210000       7.780000   
50%       283.470000       5.220000      79.300000 